In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing, SimpleExpSmoothing
from sklearn import linear_model

In [3]:
#Load Base Dataser
demo = pd.read_csv('demographics.csv')
pop = pd.read_csv('pop.csv')
demo.head()

,Year,Country of origin,Country of origin (ISO),Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,Female total,Male 0 - 4,Male 5 - 11,Male 12 - 17,Male 18 - 59,Male 60,Male total,Total
0,2011,Afghanistan,AFG,203989,20,141,661973,39839,1520434,214818,10,188,690934,68179,1634982,4064481
1,2011,Antigua and Barbuda,ATG,0,0,0,6,0,6,0,0,0,6,0,6,32
2,2011,Azerbaijan,AZE,27558,5,5,172575,35036,310900,27976,5,0,164055,34770,299230,615364
3,2011,Burundi,BDI,17317,0,40,54232,4126,169244,17495,0,44,53300,4114,162975,342577
4,2011,Burkina Faso,BFA,5,0,0,55,0,115,5,0,0,140,0,216,1262


In [4]:
demo_scaled = demo.join(pop.set_index('Country Code'), on = 'Country of origin (ISO)')

In [5]:
df = demo_scaled[['Year', 'Country of origin','Country of origin (ISO)', 'Female total','Male total', 'Total', '2019']]
df.head()

,Year,Country of origin,Country of origin (ISO),Female total,Male total,Total,2019
0,2011,Afghanistan,AFG,1520434,1634982,4064481,38041754.0
1,2011,Antigua and Barbuda,ATG,6,6,32,97118.0
2,2011,Azerbaijan,AZE,310900,299230,615364,10023318.0
3,2011,Burundi,BDI,169244,162975,342577,11530580.0
4,2011,Burkina Faso,BFA,115,216,1262,20321378.0


In [6]:
demo_scaled['female'] = demo_scaled['Female total'] / (demo_scaled['2019']/1000000)
demo_scaled['male'] = demo_scaled['Male total'] / (demo_scaled['2019']/1000000)
demo_scaled['total'] = demo_scaled['Total'] / (demo_scaled['2019']/1000000)


demo_scaled = demo_scaled.rename(columns={'2019':'pop'})
demo_scaled.head()

,Year,Country of origin,Country of origin (ISO),Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,Female total,Male 0 - 4,...,Male 12 - 17,Male 18 - 59,Male 60,Male total,Total,Country Name,pop,female,male,total
0,2011,Afghanistan,AFG,203989,20,141,661973,39839,1520434,214818,...,188,690934,68179,1634982,4064481,Afghanistan,38041754.0,39967.505179,42978.617653,106842.628760
1,2011,Antigua and Barbuda,ATG,0,0,0,6,0,6,0,...,0,6,0,6,32,Antigua and Barbuda,97118.0,61.780514,61.780514,329.496077
2,2011,Azerbaijan,AZE,27558,5,5,172575,35036,310900,27976,...,0,164055,34770,299230,615364,Azerbaijan,10023318.0,31017.672990,29853.387870,61393.243235
3,2011,Burundi,BDI,17317,0,40,54232,4126,169244,17495,...,44,53300,4114,162975,342577,Burundi,11530580.0,14677.839276,14134.154570,29710.300783
4,2011,Burkina Faso,BFA,5,0,0,55,0,115,5,...,0,140,0,216,1262,Burkina Faso,20321378.0,5.659065,10.629200,62.102088


In [7]:
PPP = pd.read_csv('PPP.csv')
PPP.head()

,ISO,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,ABW,24985.993280,24713.698050,26189.435510,26647.938100,27980.880700,28281.350480,29007.693000,NaN,NaN
1,AFG,591.162759,641.871479,637.165523,613.856689,578.466353,509.218661,519.884773,493.750418,507.103432
2,AGO,4615.468028,5100.095808,5254.882338,5408.410496,4166.979684,3506.072885,4095.812942,3289.646664,2790.726615
3,ALB,4437.142885,4247.629984,4413.060861,4578.631994,3952.801215,4124.055726,4531.020806,5284.380184,5353.244856
4,AND,43335.328860,38686.461260,39538.766720,41303.929370,35762.523070,37474.665410,38962.880350,41793.055260,40886.391160


In [68]:
afg_ppp = PPP[PPP['ISO'] == 'AFG'].drop(columns=PPP.columns[0]).T
afg_ppp=np.array(afg_ppp)
afg_ppp

array([[591.162759 ],
       [641.8714792],
       [637.1655232],
       [613.8566892],
       [578.4663529],
       [509.2186613],
       [519.8847731],
       [493.7504181],
       [507.1034319]])

In [69]:
GDP = pd.read_csv('GDP.csv')
GDP.head()

,Country Name,ISO,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,NaN,NaN
1,Afghanistan,AFG,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10
2,Angola,AGO,1.117900e+11,1.280530e+11,1.367100e+11,1.457120e+11,1.161940e+11,1.011240e+11,1.221240e+11,1.013530e+11,8.881570e+10
3,Albania,ALB,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527918e+10
4,Andorra,AND,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09


In [70]:
afg_gdp = GDP[GDP['ISO'] == 'AFG'].drop(columns=GDP.columns[[0,1]]).T
afg_gdp=np.array(afg_gdp)
afg_gdp

array([[1.78042930e+10],
       [2.00015985e+10],
       [2.05610696e+10],
       [2.04848851e+10],
       [1.99071114e+10],
       [1.80177491e+10],
       [1.88699457e+10],
       [1.83538811e+10],
       [1.92911040e+10]])

In [71]:
life = pd.read_csv('life expectancy.csv')
life.head()

,Country Name,ISO,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,75.017,75.158,75.299,75.441,75.583,75.725,75.868,76.010,76.152
1,Afghanistan,AFG,61.028,61.553,62.054,62.525,62.966,63.377,63.763,64.130,64.486
2,Angola,AGO,55.350,56.330,57.236,58.054,58.776,59.398,59.925,60.379,60.782
3,Albania,ALB,76.562,76.914,77.252,77.554,77.813,78.025,78.194,78.333,78.458
4,Andorra,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
afg_life = life[life['ISO'] == 'AFG'].drop(columns=life.columns[[0,1]]).T
afg_life = np.array(afg_life)
afg_life

array([[61.028],
       [61.553],
       [62.054],
       [62.525],
       [62.966],
       [63.377],
       [63.763],
       [64.13 ],
       [64.486]])

In [73]:
df2 = demo_scaled[['Country of origin (ISO)','Year','total']]
df2.head()

,Country of origin (ISO),Year,total
0,AFG,2011,106842.628760
1,ATG,2011,329.496077
2,AZE,2011,61393.243235
3,BDI,2011,29710.300783
4,BFA,2011,62.102088


In [89]:
afg = df2[df2['Country of origin (ISO)'] == 'AFG']
afg.head()


,Country of origin (ISO),Year,total
0,AFG,2011,106842.628760
25,AFG,2012,103909.088945
50,AFG,2013,91046.590544
75,AFG,2014,94712.194396
100,AFG,2015,104910.041740


In [90]:
afg['ppp'] = afg_ppp
afg['gdp'] = afg_gdp
afg['life'] = afg_life
afg

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

,Country of origin (ISO),Year,total,ppp,gdp,life
0,AFG,2011,106842.628760,591.162759,1.780429e+10,61.028
25,AFG,2012,103909.088945,641.871479,2.000160e+10,61.553
50,AFG,2013,91046.590544,637.165523,2.056107e+10,62.054
75,AFG,2014,94712.194396,613.856689,2.048489e+10,62.525
100,AFG,2015,104910.041740,578.466353,1.990711e+10,62.966
125,AFG,2016,116009.608810,509.218661,1.801775e+10,63.377
150,AFG,2017,129052.487958,519.884773,1.886995e+10,63.763
175,AFG,2018,138742.524858,493.750418,1.835388e+10,64.130
200,AFG,2019,150666.475578,507.103432,1.929110e+10,64.486


In [91]:
ab_afg = afg.drop(columns=afg.columns[0])
ab_afg

,Year,total,ppp,gdp,life
0,2011,106842.628760,591.162759,1.780429e+10,61.028
25,2012,103909.088945,641.871479,2.000160e+10,61.553
50,2013,91046.590544,637.165523,2.056107e+10,62.054
75,2014,94712.194396,613.856689,2.048489e+10,62.525
100,2015,104910.041740,578.466353,1.990711e+10,62.966
125,2016,116009.608810,509.218661,1.801775e+10,63.377
150,2017,129052.487958,519.884773,1.886995e+10,63.763
175,2018,138742.524858,493.750418,1.835388e+10,64.130
200,2019,150666.475578,507.103432,1.929110e+10,64.486


In [92]:
X = ab_afg.drop(columns='total')
model1 = sm.OLS(ab_afg['total'], sm.add_constant(X))

model1 = model1.fit()
model1.summary()

D:\anaconda\lib\site-packages\scipy\stats\stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  total   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                     46.61
Date:                Thu, 20 May 2021   Prob (F-statistic):            0.00131
Time:                        23:04:38   Log-Likelihood:                -84.142
No. Observations:                   9   AIC:                             178.3
Df Residuals:                       4   BIC:                             179.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -9.567e+07   1.87e+07     -5.106      0.007   -1.48e+08   -4.36e+07
Year        5.031e+04   1.05e+04      4.790      0.009    2.11e+04    7.95e+04
ppp          394.8072    780.751      0.506      0.640   -1772.906    2562.521
gdp        -1.478e-05   2.39e-05     -0.618      0.570   -8.12e-05    5.16e-05
life       -8.791e+04   4.71e+04     -1.866      0.135   -2.19e+05    4.29e+04
==============================================================================
Omnibus:                        0.388   Durbin-Watson:                   2.989
Prob(Omnibus):                  0.824   Jarque-Bera (JB):                0.465
Skew:                           0.260   Prob(JB):                        0.793
Kurtosis:                       2.015   Cond. No.                     2.60e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.6e+14. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

ab_afg['is_treated'] = (np.random.rand(9)<0.5).astype(int)
ab_afg['total'] = (ab_afg['is_treated'] * 1.2)*ab_afg['ppp']

model1 = sm.OLS(ab_afg['total'], sm.add_constant(ab_afg['treated']))
model1 = model1.fit()
model1.summary()

In [95]:
X_train,X_test,y_train,y_test = train_test_split(X,ab_afg['total'],random_state=10)

In [93]:
clf = linear_model.Lasso(alpha=0.05)

In [96]:
clf.fit(X_train, y_train)
clf.score(X_train, y_train), clf.score(X_test, y_test)

D:\anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


(0.9830050264213505, 0.8311751049122802)

In [107]:
def myLasso(country):
    df_ppp = PPP[PPP['ISO'] == country].drop(columns=PPP.columns[0]).T
    df_ppp=np.array(df_ppp)
    
    df_life = life[life['ISO'] == country].drop(columns=life.columns[[0,1]]).T
    df_life=np.array(df_life)
    
    df_gdp = GDP[GDP['ISO'] == country].drop(columns=GDP.columns[[0,1]]).T
    df_gdp=np.array(df_gdp)
    
    df = df2[df2['Country of origin (ISO)'] == country]
    df['ppp'] = df_ppp
    df['gdp'] = df_gdp
    df['life'] = df_life
    X_train,X_test,y_train,y_test = train_test_split(X,df['total'],random_state=10)
    clf = linear_model.Lasso(alpha=0.05)
    clf.fit(X_train, y_train)
    return clf.score(X_train, y_train), clf.score(X_test, y_test)

In [108]:
myLasso('AFG')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
D:\anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\anaconda\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

(0.9830050264213505, 0.8311751049122802)

In [100]:
myLasso('COL')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
D:\anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\anaconda\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

(0.9995848068635492, 0.9401093803739301)